In [1]:
%%writefile lab/PSO.cpp

#include <mpi.h>
#include <iostream>
#include <omp.h>
#include <cstdlib>
#include <time.h>
#include <random>
#include <thread>
#include <math.h>
using namespace std;

constexpr int kMaster = 0;
const int n_dim = 6;
const int n_particles_all = 16000;
const float solution_space = 15;
const int iters = 5000;
const float w = 0.8;
const float c1 = 0.8;
const float c2 = 0.8;


float rand_num(float min = 0.0, float max = 1.0, int seed = 0);
static float search_parrallel(float* x, float* v, float* pbest, float* gbest, int k, int num_procs, int rank);


float func(float x[])
{
    float y;
    y = pow((x[0] - 2), 2) + pow((x[1] - 5), 2) + pow((x[2] - 8), 2) + pow((x[3] + 3.2425), 2) + pow((x[4] - 2.43), 2) + pow((x[5] - 3), 2);
    return y;
}


int main(int argc, char* argv[]) {
    int id, num_procs;
    MPI_Status stat;
    const clock_t begin_time = clock();
    
    
    // Start MPI.
    if (MPI_Init(&argc, &argv) != MPI_SUCCESS) {
        cout << "Failed to initialize MPI\n";
        exit(-1);
    }
    // Create the communicator, and retrieve the number of processes.
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    // Determine the rank of the process.
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    const int device = id % 2;
    
    
    //initialization
    const int n_particles = n_particles_all / num_procs;
    const int N = n_particles * n_dim;
    float* x = new float [N];
    float* v = new float [N];
    for (int i = 0; i < N; i++)
    {
        x[i] = rand_num(-solution_space, solution_space, id);
        v[i] = rand_num();
    }
    float pbest_f = func(&x[0]), gbest_f = func(&x[0]);
    float* pbest = new float[n_dim];
    float* gbest = new float[n_dim];

    
    //start search
    //#pragma omp target data device(device) map(to:pbest[0:n_dim], gbest[0:n_dim]) map(tofrom:v[0:N], x[0:N])
    for (int k = 1; k <= iters; k++)
    {
        //particles move and get best of current iteration
        gbest_f = search_parrallel(x, v, pbest, gbest, k, num_procs, device);
        //initialize 2d array for best f and particle of each thread
        float* proc_f = nullptr;
        float* proc_x = nullptr;
        if (id == kMaster)
        {
            proc_f = new float[num_procs];
            proc_x = new float[num_procs * n_dim];
        }
        MPI_Gather(&gbest_f, 1, MPI_FLOAT, proc_f, 1, MPI_FLOAT, 0, MPI_COMM_WORLD);
        MPI_Gather(gbest, n_dim, MPI_FLOAT, proc_x, n_dim, MPI_FLOAT, 0, MPI_COMM_WORLD);
        //get best of current iteration
        if (id == kMaster)
        {
            for (int i = 0; i < num_procs; i++)
            {
                if (proc_f[i] < gbest_f)
                {
                    gbest_f = proc_f[i];
                    copy(&proc_x[i * n_dim], &proc_x[(i + 1) * n_dim], gbest);
                }
            }
            //get best of all iterations
            if (gbest_f < pbest_f)
            {
                pbest_f = gbest_f;
                copy(gbest, gbest + n_dim, pbest);
            }
            delete[] proc_f;
            delete[] proc_x;
        }
        MPI_Bcast(&gbest_f, 1, MPI_FLOAT, 0, MPI_COMM_WORLD);
        MPI_Bcast(gbest, n_dim, MPI_FLOAT, 0, MPI_COMM_WORLD);
        MPI_Bcast(&pbest_f, 1, MPI_FLOAT, 0, MPI_COMM_WORLD);
        MPI_Bcast(pbest, n_dim, MPI_FLOAT, 0, MPI_COMM_WORLD);
    }
    
    
    //print answer
    if (id == kMaster)
    {
        cout << "solution : [";
        for (int j = 0; j < n_dim; j++)
        {
            if (j!=0)
                cout << ", ";
            cout << pbest[j];
        }
        cout << "]" << endl;
        cout << pbest_f << endl;
        cout << "Time spent: " << float( clock () - begin_time ) /  CLOCKS_PER_SEC << " secs" << endl;
    }

    
    //free memory
    MPI_Finalize();
    delete[] x;
    delete[] v;
    delete[] pbest;
    delete[] gbest;

    return 0;
}

float rand_num(float min, float max, int seed)
{
    static thread_local mt19937 generator(time(0) + seed);
    //static thread_local mt19937 generator;
    uniform_real_distribution<double> distribution(min, max);
    return distribution(generator);
}

static float search_parrallel(float* x, float* v, float* pbest, float* gbest, int k, int num_procs, int rank)
{
    static char machine_name[MPI_MAX_PROCESSOR_NAME];
    static int name_len;
    static int is_cpu=true;
    
    static int n_particles = n_particles_all / num_procs;
    static int N = n_particles * n_dim;
    
    //particles relocate
    float new_gbest_f;
    int new_gbest_id;
    float r1 = rand_num(), r2 = rand_num();
    if (k == 1)
    {
        copy(&x[new_gbest_id], &x[new_gbest_id + n_dim], gbest);
    }
    //#pragma omp target update device(rank) to(pbest[0:n_dim], gbest[0:n_dim])
    #pragma omp target map(from:is_cpu) map(to:pbest[0:n_dim], gbest[0:n_dim]) map(tofrom:v[0:N], x[0:N])
    {
        //#pragma target device(rank) omp teams distribute parallel for simd
        #pragma omp teams distribute parallel for simd
        for (int i = 0; i < N; i++)
        {
            int j = i % n_dim;
            if (i==0) is_cpu = omp_is_initial_device();
            if (k == 1)
            {
                v[i] = w * v[i];
            }
            else
            {
                v[i] = w * v[i] + c1 * r1 * (pbest[j] - x[i]) + c2 * r2 * (gbest[j] - x[i]);
            }
            x[i] += v[i];
        }
    }
    for (int i = 0; i < n_particles; i++)
    {
        if ((i == 0) || (func(&x[i * n_dim]) < new_gbest_f))
        {
            new_gbest_f = func(&x[i * n_dim]);
            new_gbest_id = i;
        }
    }
    copy(&x[new_gbest_id * n_dim], &x[(new_gbest_id + 1) * n_dim], gbest);
    if (k == 1)
    {
        MPI_Get_processor_name(machine_name, &name_len);
        cout << " runs on: " << machine_name << ", uses device: " << (is_cpu?"CPU":"GPU") << endl;
    }
    return new_gbest_f;
}

Overwriting lab/PSO.cpp


In [2]:
%pycat compile_omp_c.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is compiling
mpiicpc -cxx=icpx lab/PSO.cpp -fiopenmp -fopenmp-targets=spir64 -o bin/PSO.x


The following cell will submit the execution of the compilation script using the __q__ script. The __q__ script submits jobs to the DevCloud and retrieves the output. The first arguments to __q__ is the script to execute. The second argument is the properties of the nodes to request. In the following cell, we're requesting one node with the property ppn=2.

In [3]:
! chmod 755 q; chmod 755 compile_omp_c.sh; ./q compile_omp_c.sh nodes=1:ppn=2

Job has been submitted to Intel(R) DevCloud with  nodes and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2182726.v-qsvr-1           ...ub-singleuser u184460         00:04:01 R jupyterhub     
2183801.v-qsvr-1           compile_omp_c.sh u184460                0 Q batch          

Waiting for Output ████████████ Done⬇

########################################################################
#      Date:           Thu 09 Feb 2023 07:54:39 AM PST
#    Job ID:           2183801.v-qsvr-1.aidevcloud
#      User:           u184460
# Resources:           cput=75:00:00,neednodes=1:ppn=2,nodes=1:ppn=2,walltime=06:00:00
########################################################################

## u184460 is compiling

#########

### Execute the Code
Next we will execute the compiled binary with the __mpirun__ command to launch the MPI job using 4 processes. Examine the launch script by executing the following cell.

In [4]:
%pycat launch.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
% setenv OMP_TARGET_OFFLOAD MANDATORY
export IBOMPTARGET_INFO=4
/bin/echo "##" $(whoami) is executing
mpirun -np 4 bin/PSO.x


Execute the following cell to run the program on multiple nodes. In the following example, 2 nodes with GPUs are requested.

In [5]:
! chmod 755 q; chmod 755 launch.sh; ./q launch.sh nodes=2:gpu:ppn=2

Job has been submitted to Intel(R) DevCloud with  nodes and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2182726.v-qsvr-1           ...ub-singleuser u184460         00:04:01 R jupyterhub     
2183802.v-qsvr-1           launch.sh        u184460                0 Q batch          

Waiting for Output ███████████████████ Done⬇

########################################################################
#      Date:           Thu 09 Feb 2023 07:54:51 AM PST
#    Job ID:           2183802.v-qsvr-1.aidevcloud
#      User:           u184460
# Resources:           cput=75:00:00,neednodes=2:gpu:ppn=2,nodes=2:gpu:ppn=2,walltime=06:00:00
########################################################################

## u184460 is execu